In [ ]:
!pip install streamlit playwright feedparser


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 4.5 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6046 sha256=b7fe0b59505ec2b9eb5bef169c0875e927b43fc7b49f979d141d9a7796b5ea85
  Stored in directory: /root/.cache/pip/wheels/3b/25/2a/105d6a15df6914f4d15047691c6c28f9052cc1173e40285d03
Successfully built sgmllib3k


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time
from urllib.parse import urljoin, urlparse
import streamlit as st
from urllib.robotparser import RobotFileParser
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import json

#Member 1 - Crawlability Specialist


In [ ]:
BASE_URL = "https://www.aljazeera.com"

In [ ]:
def analyze_robots_txt(base_url, user_agent='*', target_path=None):
    parsed_url = urlparse(base_url)
    robots_url = f"{parsed_url.scheme}://{parsed_url.netloc}/robots.txt"

    try:
        response = requests.get(robots_url, timeout=10)
        if response.status_code != 200:
            return {
                "status": "robots.txt not found or inaccessible.",
                "robots_url": robots_url,
                "allowed": None,
                "rules": {},
                "sitemaps": [],
                "crawl_delay": None
            }

        robots_txt = response.text

        rp = RobotFileParser()
        rp.set_url(robots_url)
        rp.read()

        lines = robots_txt.splitlines()
        sitemaps = []
        crawl_delay = None
        allow_rules = []
        disallow_rules = []

        applicable = False
        for line in lines:
            line = line.strip()
            if not line or line.startswith("#"):
                continue

            if line.lower().startswith("user-agent:"):
                ua = line.split(":")[1].strip()
                applicable = (ua == "*" or ua.lower() == user_agent.lower())

            if applicable:
                if line.lower().startswith("allow:"):
                    allow_rules.append(line.split(":", 1)[1].strip())
                elif line.lower().startswith("disallow:"):
                    disallow_rules.append(line.split(":", 1)[1].strip())
                elif line.lower().startswith("crawl-delay:"):
                    try:
                        crawl_delay = float(line.split(":", 1)[1].strip())
                    except ValueError:
                        pass

            if line.lower().startswith("sitemap:"):
                sitemaps.append(line.split(":", 1)[1].strip())
        allowed = None
        if target_path:
            full_url = urljoin(base_url, target_path)
            allowed = rp.can_fetch(user_agent, full_url)

        return {
            "status": "robots.txt analyzed successfully",
            "robots_url": robots_url,
            "allowed": allowed,
            "rules": {
                "allow": allow_rules,
                "disallow": disallow_rules
            },
            "sitemaps": sitemaps,
            "crawl_delay": crawl_delay
        }

    except Exception as e:
        return {
            "status": f"Error occurred: {e}",
            "robots_url": robots_url,
            "allowed": None,
            "rules": {},
            "sitemaps": [],
            "crawl_delay": None
        }
def print_robots_analysis(result):
      print("\n" + "=" * 60)
      print("🔍 robots.txt Analysis Summary")
      print("=" * 60)
      print(f"URL: {result['robots_url']}")
      print(f"Status: {result['status']}")
      print(f"Crawling Allowed: {'✅ Yes' if result['allowed'] else '❌ No' if result['allowed'] is not None else 'Unknown'}")

      print("\n📜 Crawl Rules for User-Agent '*':")
      allow = result.get('rules', {}).get('allow', [])
      disallow = result.get('rules', {}).get('disallow', [])

      print("\n✅ Allowed Paths:")
      if allow:
          for path in allow:
              print(f"  - {path}")
      else:
          print("  None")

      print("\n❌ Disallowed Paths:")
      if disallow:
          for path in disallow:
              print(f"  - {path}")
      else:
          print("  None")

      print("\n🕓 Crawl Delay:")
      delay = result.get('crawl_delay')
      print(f"  {delay if delay is not None else 'Not specified'}")

      print("\n🗺️ Sitemap Links:")
      sitemaps = result.get('sitemaps', [])
      if sitemaps:
          for idx, sitemap in enumerate(sitemaps, 1):
              print(f"  {idx}. {sitemap}")
      else:
          print("  None")
      print("=" * 60 + "\n")


In [ ]:
result = analyze_robots_txt(BASE_URL, target_path="/about")
print_robots_analysis(result)


🔍 robots.txt Analysis Summary
URL: https://www.aljazeera.net/robots.txt
Status: robots.txt analyzed successfully
Crawling Allowed: ✅ Yes

📜 Crawl Rules for User-Agent '*':

✅ Allowed Paths:
  - /search/$

❌ Disallowed Paths:
  - /api
  - /asset-manifest.json
  - /search/
  - /home/search?q=

🕓 Crawl Delay:
  Not specified

🗺️ Sitemap Links:
  1. https://www.aljazeera.net/sitemap.xml
  2. https://www.aljazeera.net/news-sitemap.xml
  3. https://www.aljazeera.net/sitemaps/article-archive.xml
  4. https://www.aljazeera.net/sitemaps/article-new.xml
  5. https://www.aljazeera.net/sitemaps/video-archive.xml
  6. https://www.aljazeera.net/sitemaps/video-new.xml



#Member 2 – Content Extractor

In [ ]:
!pip install selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 22.0 MB/s eta 0:00:00


In [56]:
import time
import requests
import re
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime

HEADERS = {
    'User-Agent': 'Mozilla/5.0'
}

def fetch_page(url):
    response = requests.get(url, headers=HEADERS, timeout=10)
    response.raise_for_status()
    return response.text

def extract_publish_date(soup):
    time_tag = soup.find("time")
    if time_tag and time_tag.has_attr('datetime'):
        return time_tag['datetime']

    date_pattern = re.compile(r'\d{1,2}/\d{1,2}/\d{4}')
    for span in soup.find_all("span"):
        text = span.get_text(strip=True)
        if date_pattern.search(text):
            try:
                return datetime.strptime(text, "%d/%m/%Y").date().isoformat()
            except ValueError:
                return text
    return None

def extract_article_image(soup, base_url):
    image_div = soup.find("div", class_="responsive-image")
    if image_div:
        img_tag = image_div.find("img")
        if img_tag and img_tag.has_attr("src"):
            return urljoin(base_url, img_tag["src"])
    return None

def extract_article_paragraphs(soup):
    paragraphs = soup.find_all("p")
    clean_paragraphs = [p.get_text(strip=True) for p in paragraphs if p.get_text(strip=True)]
    return clean_paragraphs

def parse_article_details(article_url):
    try:
        html = fetch_page(article_url)
        soup = BeautifulSoup(html, 'html.parser')

        summary_tag = soup.select_one("meta[name='description']")
        summary = summary_tag['content'] if summary_tag else None

        publish_date = extract_publish_date(soup)

        category_tag = soup.select_one("a[href*='/news/'][class*='Topic']") \
                       or soup.select_one("div.topic a") \
                       or soup.select_one("a[href*='/news/']")
        category = category_tag.get_text(strip=True) if category_tag else None

        image_url = extract_article_image(soup, base_url="https://www.aljazeera.net")
        paragraphs = extract_article_paragraphs(soup)

        return {
            "summary": summary,
            "published_at": publish_date,
            "category": category,
            "image": image_url,
            "paragraphs": paragraphs
        }

    except Exception as e:
        print(f"Failed to parse article {article_url}: {e}")
        return {}

def extract_articles_from_selenium(driver, base_url):
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    article_links = soup.select("article a[href^='/news']")
    seen_links = set()
    articles = []

    print(f"\n📄 Found {len(article_links)} article links.")
    for tag in article_links:
        link = urljoin(base_url, tag.get("href"))
        title = tag.get_text(strip=True)

        if link in seen_links or not title:
            continue

        seen_links.add(link)
        print(f"  -> Fetching article: {link}")
        details = parse_article_details(link)
        if not details:
            continue

        article_info = {
            "title": title,
            "link": link,
            **details
        }
        articles.append(article_info)
        time.sleep(1)

    return articles

def click_show_more(driver, max_clicks=100):
    clicks = 0
    while clicks < max_clicks:
        try:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)

            show_more_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, "//*[@id='news-feed-container']/button"))
            )

            driver.execute_script("arguments[0].scrollIntoView(true);", show_more_button)
            time.sleep(1)

            driver.execute_script("arguments[0].click();", show_more_button)

            clicks += 1
            print(f"✅ Clicked 'عرض المزيد' ({clicks})")
            time.sleep(4)

        except Exception as e:
            print(f"❌ No more 'عرض المزيد' or error: {e}")
            break

    print(f"🔁 Finished clicking 'Show More'. Total clicks: {clicks}")

def main():
    base_url = "https://www.aljazeera.com/news/"
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--no-sandbox")

    driver = webdriver.Chrome(options=chrome_options)
    driver.get(base_url)
    time.sleep(3)

    click_show_more(driver, max_clicks=100)

    articles = extract_articles_from_selenium(driver, base_url)
    driver.quit()

    print(f"\n📰 Total articles collected: {len(articles)}\n")
    for idx, article in enumerate(articles, 1):
        print(f"\n🖨️ Printing article {idx}/{len(articles)}: {article['title']}")
        print(article)
    save_articles_to_csv(articles)
    generate_csv_summary("aljazeera_articles.csv")

if __name__ == "__main__":
    main()


✅ Clicked 'عرض المزيد' (1)
✅ Clicked 'عرض المزيد' (2)
✅ Clicked 'عرض المزيد' (3)
✅ Clicked 'عرض المزيد' (4)
✅ Clicked 'عرض المزيد' (5)
✅ Clicked 'عرض المزيد' (6)
✅ Clicked 'عرض المزيد' (7)
✅ Clicked 'عرض المزيد' (8)
✅ Clicked 'عرض المزيد' (9)
✅ Clicked 'عرض المزيد' (10)
✅ Clicked 'عرض المزيد' (11)
✅ Clicked 'عرض المزيد' (12)
✅ Clicked 'عرض المزيد' (13)
✅ Clicked 'عرض المزيد' (14)
✅ Clicked 'عرض المزيد' (15)
✅ Clicked 'عرض المزيد' (16)
✅ Clicked 'عرض المزيد' (17)
✅ Clicked 'عرض المزيد' (18)
✅ Clicked 'عرض المزيد' (19)
✅ Clicked 'عرض المزيد' (20)
✅ Clicked 'عرض المزيد' (21)
✅ Clicked 'عرض المزيد' (22)
✅ Clicked 'عرض المزيد' (23)
✅ Clicked 'عرض المزيد' (24)
✅ Clicked 'عرض المزيد' (25)
✅ Clicked 'عرض المزيد' (26)
✅ Clicked 'عرض المزيد' (27)
✅ Clicked 'عرض المزيد' (28)
✅ Clicked 'عرض المزيد' (29)
✅ Clicked 'عرض المزيد' (30)
✅ Clicked 'عرض المزيد' (31)
✅ Clicked 'عرض المزيد' (32)
✅ Clicked 'عرض المزيد' (33)
✅ Clicked 'عرض المزيد' (34)
✅ Clicked 'عرض المزيد' (35)
✅ Clicked 'عرض المزيد' (36)
✅

In [53]:
import pandas as pd
from collections import Counter
from datetime import datetime
import re

def generate_csv_summary(csv_file="aljazeera_articles.csv"):
    try:
        df = pd.read_csv(csv_file)

        # Basic cleanup
        df["published_at"] = pd.to_datetime(df["published_at"], errors='coerce')
        df["paragraphs"] = df["paragraphs"].fillna("")

        total_articles = len(df)
        date_range = f"{df['published_at'].min().date()} to {df['published_at'].max().date()}" if not df['published_at'].isna().all() else "Unknown"

        top_categories = df["category"].value_counts().head(5).to_dict()

        # Tokenize and clean title words
        title_words = " ".join(df["title"].dropna().tolist()).lower()
        title_words = re.findall(r'\b\w{4,}\b', title_words)  # only words with 4+ characters
        common_keywords = Counter(title_words).most_common(5)

        longest_article = df.loc[df["paragraphs"].str.len().idxmax()]

        summary = f"""
📊 Al Jazeera English Article Summary
------------------------------------
📰 Total articles: {total_articles}
🗓️ Date range: {date_range}

🏷️ Top Categories:
""" + "\n".join([f"   - {cat}: {count}" for cat, count in top_categories.items()]) + """

🔑 Top Keywords in Titles:
""" + "\n".join([f"   - {word}: {count}" for word, count in common_keywords]) + f"""

📝 Longest Article:
   - Title: {longest_article['title']}
   - Link: {longest_article['link']}
   - Word Count: {len(longest_article['paragraphs'].split())}
"""

        print(summary)

        # Optional: Save summary to a .txt file
        with open("aljazeera_summary.txt", "w", encoding="utf-8") as f:
            f.write(summary)

        print("✅ Summary saved to aljazeera_summary.txt")

    except Exception as e:
        print(f"❌ Error generating summary: {e}")


In [52]:
import csv

def save_articles_to_csv(articles, filename="aljazeera_articles.csv"):
    with open(filename, mode='w', newline='', encoding='utf-8-sig') as file:  # UTF-8 with BOM
        writer = csv.DictWriter(file, fieldnames=[
            "title", "link", "summary", "published_at", "category", "image", "paragraphs"
        ])
        writer.writeheader()
        for article in articles:
            writer.writerow({
                "title": article.get("title"),
                "link": article.get("link"),
                "summary": article.get("summary"),
                "published_at": article.get("published_at"),
                "category": article.get("category"),
                "image": article.get("image"),
                "paragraphs": " ".join(article.get("paragraphs", []))
            })
    print(f"\n💾 تم حفظ المقالات في: {filename}")


#Member 3 – JS & API Handler

In [ ]:
import requests
from bs4 import BeautifulSoup

url = "https://www.aljazeera.net"
res = requests.get(url)
soup = BeautifulSoup(res.text, 'html.parser')
print(soup.get_text()[:1000])  # Inspect manually


الجزيرة نت: آخر أخبار اليوم حول العالم

تخطي الروابطاذهب إلى المحتوى المميّزاذهب إلى تغذية المحتوىانتقل إلى الأكثر قراءةرابط إلى الصفحة الرئيسية للجزيرةplay البث الحي تسجيلأظهر القائمة الرئيسيةNavigation menuأخبارالآناعرض المزيدسوريا بعد الأسدطوفان الأقصىالأزمة السودانيةحرب أوكرانياأفريقياأبعادرياضةمقالاتبيئةاقتصادثقافةفيديوالمزيداعرض المزيدسفرصحةسياسةتكنولوجيامدوناتالموسوعةتراثفنعلومأسلوب حياةمعمّقةالقدسأسرةمنوعاتحرياتإذاعةبالصورplay البث الحي اضغط هنا للبحثsearchتسجيلNavigation menucaret-leftالآنسوريافلسطينالسودانأوكرانياcaret-rightمحتوى رئيسيaj-logoقد يحتوي هذا الفيديو أنماطا ضوئية أو صورا يمكن أن تسبب نوبات صرع أو عدم ارتياح لمن يعاني حساسيات بصريةالحرب على غزة.. حماس ترفض شروط نتنياهو وتنديد أوروبي بإسرائيلblinking-dotتغطية مباشرةتغطية مباشرة, list 1 of 10live-blueنشر منذ 42 دقائقمنذ 42د93 شهيدا في قطاع غزة منذ فجر الأربعاءlist 2 of 10live-blueنشر منذ 44 دقائقمنذ 44دالاحتلال يستهدف مستشفى العودة شمال غزةإسرائيل: اعتراض صاروخ أُطلق من اليمنأعلنت الجبهة الداخلية الإسرائيلية تفعيل صف

In [ ]:
import subprocess
subprocess.run(["playwright", "install"], check=True)


CompletedProcess(args=['playwright', 'install'], returncode=0)

In [ ]:
import nest_asyncio
import asyncio
from playwright.async_api import async_playwright

nest_asyncio.apply()

async def fetch_with_playwright_async(url):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        await page.goto(url, timeout=30000)
        content = await page.content()
        await browser.close()
        return content


if __name__ == "__main__":
    url = "https://www.aljazeera.net/news"
    html_content = asyncio.get_event_loop().run_until_complete(fetch_with_playwright_async(url))
    print(html_content[:1000])


<!DOCTYPE html><html lang="ar" dir="rtl" class="theme-aja" style="--vh: 7.2px; --clientWidth: 1280px;"><head><meta charset="utf-8"><meta name="viewport" content="width=device-width,initial-scale=1,shrink-to-fit=no"><meta http-equiv="content-type" content="text/html; charset=UTF-8"><meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1"><link rel="shortcut icon" href="/favicon_aja.ico"><title data-reactroot="">أخبار العالم | الجزيرة نت</title><script async="" src="https://connect.facebook.net/en_US/fbevents.js"></script><script type="text/javascript" async="" src="https://www.googletagmanager.com/gtag/js?id=AW-947178488&amp;cx=c&amp;gtm=45je55k1h1v894171536za200&amp;tag_exp=101509157~103116026~103130495~103130497~103136993~103136995~103200004~103233427~103252644~103252646~103301114~103301116"></script><script type="text/javascript" async="" src="https://static.ads-twitter.com/uwt.js"></script><script type="text/javascript" async="" src="https://secure.quantserve.com/quant.js"></sc

In [ ]:
import feedparser

rss_url = "https://www.ajnet.me/aljazeerarss/a7c186be-1baa-4bd4-9d80-a84db769f779/73d0e1b4-532f-45ef-b135-bfdff8b8cab9"
feed = feedparser.parse(rss_url)

for entry in feed.entries[:5]:
    print(f"Title: {entry.title}\nLink: {entry.link}\n")


Title: حادث "خطير" خلال تدشين سفينة حربية بكوريا الشمالية وكيم غاضب
Link: https://www.ajnet.me/news/2025/5/22/%d8%b9%d8%a7%d8%ac%d9%84-%d8%b1%d9%88%d9%8a%d8%aa%d8%b1%d8%b2-%d8%b9%d9%86-%d9%88%d9%83%d8%a7%d9%84%d8%a9-%d8%a7%d9%84%d8%a3%d9%86%d8%a8%d8%a7%d8%a1-%d8%a7%d9%84%d9%85%d8%b1%d9%83%d8%b2%d9%8a%d8%a9-2?traffic_source=rss

Title: إسرائيل: اعتراض صاروخ أُطلق من اليمن
Link: https://www.ajnet.me/news/2025/5/22/%d8%b9%d8%a7%d8%ac%d9%84-%d8%a7%d9%84%d8%ac%d8%a8%d9%87%d8%a9-%d8%a7%d9%84%d8%af%d8%a7%d8%ae%d9%84%d9%8a%d8%a9-%d8%a7%d9%84%d8%a5%d8%b3%d8%b1%d8%a7%d8%a6%d9%8a%d9%84%d9%8a%d8%a9-8?traffic_source=rss

Title: نتنياهو ينفي صحة ما يتردد عن خلاف مع ترامب
Link: https://www.ajnet.me/news/2025/5/22/%d9%86%d8%aa%d9%86%d9%8a%d8%a7%d9%87%d9%88-%d9%8a%d9%86%d9%81%d9%8a-%d8%b5%d8%ad%d8%a9-%d9%85%d8%a7-%d9%8a%d8%aa%d8%b1%d8%af%d8%af-%d8%b9%d9%86-%d8%ae%d9%84%d8%a7%d9%81-%d9%85%d8%b9?traffic_source=rss

Title: الشي الوحيد الذي أصاب ترامب فيه
Link: https://www.ajnet.me/opinions/2025/5/22/%d8%a

#Member 4 – Visual & Report Designer

In [ ]:
def calculate_crawlability_score(rules, sitemap_count=0):
    allow_count = len(rules.get('allow', []))
    disallow_count = len(rules.get('disallow', []))
    total_rules = allow_count + disallow_count

    if total_rules == 0:
        return 100  # No restrictions, fully crawlable

    # Basic ratio of allows to total rules
    allow_ratio = allow_count / total_rules

    # Give a small boost for sitemap presence
    sitemap_bonus = min(sitemap_count * 5, 20)  # max +20 points for sitemaps

    # Compute final score (0-100)
    score = int(allow_ratio * 80 + sitemap_bonus)

    return min(score, 100)


In [ ]:
import requests
from xml.etree import ElementTree as ET
from urllib.parse import urlparse

def fetch_sitemap_urls(sitemap_url):
    """Fetch URLs listed in a sitemap."""
    response = requests.get(sitemap_url)
    response.raise_for_status()
    root = ET.fromstring(response.content)
    # If it's a sitemap index (list of other sitemaps)
    loc_elements = root.findall('.//{http://www.sitemaps.org/schemas/sitemap/0.9}loc')
    return [elem.text for elem in loc_elements]

def build_sitemap_dict(urls):
    """Build sitemap dict from content URLs."""
    sitemap = {}
    for url in urls:
        parsed = urlparse(url)
        path_parts = parsed.path.strip("/").split("/")
        if len(path_parts) > 1:
            parent = path_parts[0].capitalize()
            child = path_parts[1].capitalize()
            sitemap.setdefault(parent, set()).add(child)
        elif path_parts:
            sitemap.setdefault("Home", set()).add(path_parts[0].capitalize())
    return {k: list(v) for k, v in sitemap.items()}

# Step 1: Fetch sub-sitemaps
index_sitemap_url = "https://www.aljazeera.net/sitemap.xml"
sub_sitemaps = fetch_sitemap_urls(index_sitemap_url)

# Step 2: Parse the first few sub-sitemaps to extract content URLs
content_urls = []
for sub_url in sub_sitemaps[:5]:  # Limit to avoid too many requests
    try:
        urls = fetch_sitemap_urls(sub_url)
        content_urls.extend(urls)
    except Exception as e:
        print(f"Failed to fetch {sub_url}: {e}")

# Step 3: Build the sitemap dictionary
sitemap_dict = build_sitemap_dict(content_urls[:100])  # Limit for performance

# Output result
print("Constructed sitemap dict:")
print(sitemap_dict)


Constructed sitemap dict:
{'News': ['Liveblog', '2025'], 'Politics': ['2025'], 'Videos': ['2025'], 'Opinions': ['2025'], 'Sport': ['2025'], 'Programs': ['Behindthenews'], 'Culture': ['2025'], 'Misc': ['2025'], 'Encyclopedia': ['2025'], 'Climate': ['2025'], 'Ebusiness': ['2025'], 'Science': ['2025'], 'Health': ['2025'], 'Tech': ['2025'], 'Lifestyle': ['2025'], 'Family': ['2025']}


In [ ]:
analysis = {
    'rules': {
        'allow': ['/search/$'],
        'disallow': ['/api', '/asset-manifest.json', '/search/', '/home/search?q=']
    },
    'sitemaps': [
        'https://www.aljazeera.net/sitemap.xml',
        'https://www.aljazeera.net/news-sitemap.xml',
        # etc.
    ]
}


In [ ]:
import json
import pandas as pd
from google.colab import files

# Optionally pass sitemap count if you want to use the improved function
sitemap_count = len(analysis.get('sitemaps', []))

crawlability_score = calculate_crawlability_score(analysis['rules'], sitemap_count=sitemap_count)
print("Crawlability Score:", crawlability_score)

recommendations = [
    "Use Playwright or Selenium for JS-heavy sites",
    "Check robots.txt before crawling",
    "Use Scrapy for large-scale scraping",
    "Implement retry and rate limiting"
]
# Save to JSON
data_to_save = {
    "crawlability_score": crawlability_score,
    "top_extracted_data": top_extracted_data,
    "recommendations": recommendations,
    "sitemap": sitemap_dict
}

with open("web_analysis_data.json", "w") as f:
    json.dump(data_to_save, f, indent=2)

# Also save top_extracted_data to CSV if needed
df = pd.DataFrame({"Top Extracted Data": top_extracted_data})
df.to_csv("top_extracted_data.csv", index=False)

# Download files
files.download("web_analysis_data.json")
files.download("top_extracted_data.csv")


Crawlability Score: 26


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>